In [2]:
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd

In [3]:
code_df = pd.read_csv("data/stock_code/codes.csv")

In [4]:
code_df.head()

,コード,銘柄名,市場区分,本社所在地,業務分類,決算期,売買単位,支配株主等に関する事項,注意情報等,継続企業の前提の注記
0,13010,極洋,第一部,東京,水産・農林業,3月,100,無し,,無し
1,13050,ダイワ上場投信－トピックス,ETF,-,-,-,10,-,,-
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF,-,-,-,10,-,,-
3,13080,上場インデックスファンドＴＯＰＩＸ,ETF,-,-,-,100,-,,-
4,13090,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF,-,-,-,1,-,,-


In [5]:
code_df["yahoo_code"] = (code_df["コード"]/10).astype(int).astype(str)+".T"

In [6]:
code_df.head()

,コード,銘柄名,市場区分,本社所在地,業務分類,決算期,売買単位,支配株主等に関する事項,注意情報等,継続企業の前提の注記,yahoo_code
0,13010,極洋,第一部,東京,水産・農林業,3月,100,無し,,無し,1301.T
1,13050,ダイワ上場投信－トピックス,ETF,-,-,-,10,-,,-,1305.T
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF,-,-,-,10,-,,-,1306.T
3,13080,上場インデックスファンドＴＯＰＩＸ,ETF,-,-,-,100,-,,-,1308.T
4,13090,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF,-,-,-,1,-,,-,1309.T


In [7]:
yc = code_df.yahoo_code.iloc[1]
yc

'1305.T'

In [21]:
my_share = share.Share(yc)

symbol_data = None

try:
    symbol_data = my_share.get_historical(
        share.PERIOD_TYPE_DAY, 10,
        share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)


In [27]:
stock_df = pd.DataFrame(symbol_data)
import datetime
stock_df["datetime"] = pd.to_datetime(stock_df.timestamp, unit="ms")
stock_df["code_y"] = yc
stock_df["code"] = yc[0:4]
del stock_df["timestamp"]

In [28]:
stock_df

,open,high,low,close,volume,datetime,code_y,code
0,1995.0,2001.0,1981.0,1991.0,162720,2021-07-19,1305.T,1305
1,1969.0,1981.0,1964.0,1970.0,476450,2021-07-20,1305.T,1305
2,2000.0,2009.0,1983.0,1985.0,91520,2021-07-21,1305.T,1305
3,2025.0,2026.0,2006.0,2008.0,78010,2021-07-26,1305.T,1305
4,2021.0,2026.0,2016.0,2021.0,298300,2021-07-27,1305.T,1305


## 陽線と陰線を見分ける

In [29]:
sdf = stock_df.copy()

In [60]:
sdf["o_c_diff"] = sdf.open - sdf.close
sdf["o_c_pct"] = (sdf.o_c_diff / sdf.open) * 100

In [62]:
def func_doutai(x):
  if x.o_c_diff == 0:
    return "J"
  elif x.o_c_pct > 5:
    return "D"
  else:
    return "S"
sdf["doutai"] = sdf.apply(lambda x: func_doutai(x), axis=1)
#sdf["daishou"] = abs(sdf.o_c_pct) > 5


In [63]:
sdf

,open,high,low,close,volume,datetime,code_y,code,o_c_diff,o_c_pct,daishou,doutai
0,1995.0,2001.0,1981.0,1991.0,162720,2021-07-19,1305.T,1305,4.0,0.200501,S,S
1,1969.0,1981.0,1964.0,1970.0,476450,2021-07-20,1305.T,1305,-1.0,-0.050787,S,S
2,2000.0,2009.0,1983.0,1985.0,91520,2021-07-21,1305.T,1305,15.0,0.750000,S,S
3,2025.0,2026.0,2006.0,2008.0,78010,2021-07-26,1305.T,1305,17.0,0.839506,S,S
4,2021.0,2026.0,2016.0,2021.0,298300,2021-07-27,1305.T,1305,0.0,0.000000,S,J


In [64]:
sdf["inyou"] = sdf.o_c_diff.apply(lambda x: "Y" if x > 0 else "I" if x < 0 else "J")

In [65]:
sdf

,open,high,low,close,volume,datetime,code_y,code,o_c_diff,o_c_pct,daishou,doutai,inyou
0,1995.0,2001.0,1981.0,1991.0,162720,2021-07-19,1305.T,1305,4.0,0.200501,S,S,Y
1,1969.0,1981.0,1964.0,1970.0,476450,2021-07-20,1305.T,1305,-1.0,-0.050787,S,S,I
2,2000.0,2009.0,1983.0,1985.0,91520,2021-07-21,1305.T,1305,15.0,0.750000,S,S,Y
3,2025.0,2026.0,2006.0,2008.0,78010,2021-07-26,1305.T,1305,17.0,0.839506,S,S,Y
4,2021.0,2026.0,2016.0,2021.0,298300,2021-07-27,1305.T,1305,0.0,0.000000,S,J,J
